#### Evaluate Script of Trained Models

In [1]:
from run_multimodal_time_series import *
from collections import OrderedDict
import csv

In [2]:
use_target_ratings=True

In [3]:
# loading model from saved model.
model = MultimodalEmotionPrediction()
new_state_dict = OrderedDict()
DEVICE = torch.device('cpu')   # 'cpu' in this case
if use_target_ratings:
    model_path = "../target/best_ccc_pytorch_model.bin"
else:
    model_path = "../observer/best_ccc_pytorch_model.bin"
print("loading the model from: ", model_path)
state_dict = torch.load(model_path, map_location=DEVICE)["model"]
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
_ = model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing LinguisticEncoderBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing LinguisticEncoderBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LinguisticEncoderBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


loading the model from:  ../target/best_ccc_pytorch_model.bin


In [4]:
if use_target_ratings:
    print("WARNING: use_target_ratings is setting to TRUE.")
    modality_dir_map = {"acoustic": "acoustic-egemaps",  
                        "linguistic": "linguistic-word-level", # we don't load features
                        "visual": "image-raw", # image is nested,
                        "target": "target"}
    preprocess = {
        'acoustic': lambda df : df.loc[:,' F0semitoneFrom27.5Hz_sma3nz_amean':' equivalentSoundLevel_dBp'],
        'acoustic_timer': lambda df : df.loc[:,' frameTime'],
        'linguistic': lambda df : df.loc[:,'word'],
        'linguistic_timer': lambda df : df.loc[:,'time-offset'],
        'target': lambda df : ((df.loc[:,' rating'] / 0.5) - 1.0),
        'target_timer': lambda df : df.loc[:,'time'],
    }
else:
    modality_dir_map = {"acoustic": "acoustic-egemaps",  
                        "linguistic": "linguistic-word-level", # we don't load features
                        "visual": "image-raw", # image is nested,
                        "target": "observer_EWE"}
    preprocess = {
        'acoustic': lambda df : df.loc[:,' F0semitoneFrom27.5Hz_sma3nz_amean':' equivalentSoundLevel_dBp'],
        'acoustic_timer': lambda df : df.loc[:,' frameTime'],
        'linguistic': lambda df : df.loc[:,'word'],
        'linguistic_timer': lambda df : df.loc[:,'time-offset'],
        'target': lambda df : ((df.loc[:,'evaluatorWeightedEstimate'] / 50.0) - 1.0),
        'target_timer': lambda df : df.loc[:,'time'],
    }

In [5]:
if use_target_ratings:
    output_dir = "../data-files/target/"
else:
    output_dir = "../data-files/observer/"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-uncased",
    use_fast=False,
    cache_dir="../.huggingface_cache/"
)

In [46]:
# Loading all the data partitions.
data_dir = "../../SENDv1-data/"
train_modalities_data_dir = os.path.join(data_dir, "features/Train/")
train_target_data_dir = os.path.join(data_dir, "ratings/Train")
train_SEND_features = preprocess_SEND_files(
    train_modalities_data_dir,
    train_target_data_dir,
    use_target_ratings,
    modality_dir_map=modality_dir_map,
    preprocess=preprocess,
    linguistic_tokenizer=tokenizer,
    max_number_of_file=-1
)

dev_modalities_data_dir = os.path.join(data_dir, "features/Valid/")
dev_target_data_dir = os.path.join(data_dir, "ratings/Valid")
dev_SEND_features = preprocess_SEND_files(
    dev_modalities_data_dir,
    dev_target_data_dir,
    use_target_ratings,
    modality_dir_map=modality_dir_map,
    preprocess=preprocess,
    linguistic_tokenizer=tokenizer,
    max_number_of_file=-1
)

test_modalities_data_dir = os.path.join(data_dir, "features/Test/")
test_target_data_dir = os.path.join(data_dir, "ratings/Test")
test_SEND_features = preprocess_SEND_files(
    test_modalities_data_dir,
    test_target_data_dir,
    use_target_ratings,
    modality_dir_map=modality_dir_map,
    preprocess=preprocess,
    linguistic_tokenizer=tokenizer,
    max_number_of_file=-1
)

In [47]:
# Put dataset into correct format.
train_video_id = [video_struct["video_id"] for video_struct in train_SEND_features]
train_input_a_feature = torch.stack([video_struct["a_feature"] for video_struct in train_SEND_features]).float()
train_input_l_feature = torch.stack([video_struct["l_feature"] for video_struct in train_SEND_features])
train_input_l_mask = torch.stack([video_struct["l_mask"] for video_struct in train_SEND_features])
train_input_l_segment_ids = torch.stack([video_struct["l_segment_ids"] for video_struct in train_SEND_features])
train_input_v_feature = torch.stack([video_struct["v_feature"] for video_struct in train_SEND_features]).float()
train_rating_labels = torch.stack([video_struct["rating"] for video_struct in train_SEND_features]).float()
train_seq_lens = torch.tensor([[video_struct["seq_len"]] for video_struct in train_SEND_features]).float()
train_input_mask = torch.stack([video_struct["input_mask"] for video_struct in train_SEND_features])
train_data = TensorDataset(
    train_input_a_feature, 
    train_input_l_feature, train_input_l_mask, train_input_l_segment_ids,
    train_input_v_feature, train_rating_labels, train_seq_lens, train_input_mask
)
train_dataloader = DataLoader(train_data, batch_size=1, shuffle=False)

dev_video_id = [video_struct["video_id"] for video_struct in dev_SEND_features]
dev_input_a_feature = torch.stack([video_struct["a_feature"] for video_struct in dev_SEND_features]).float()
dev_input_l_feature = torch.stack([video_struct["l_feature"] for video_struct in dev_SEND_features])
dev_input_l_mask = torch.stack([video_struct["l_mask"] for video_struct in dev_SEND_features])
dev_input_l_segment_ids = torch.stack([video_struct["l_segment_ids"] for video_struct in dev_SEND_features])
dev_input_v_feature = torch.stack([video_struct["v_feature"] for video_struct in dev_SEND_features]).float()
dev_rating_labels = torch.stack([video_struct["rating"] for video_struct in dev_SEND_features]).float()
dev_seq_lens = torch.tensor([[video_struct["seq_len"]] for video_struct in dev_SEND_features]).float()
dev_input_mask = torch.stack([video_struct["input_mask"] for video_struct in dev_SEND_features])
dev_data = TensorDataset(
    dev_input_a_feature, 
    dev_input_l_feature, dev_input_l_mask, dev_input_l_segment_ids,
    dev_input_v_feature, dev_rating_labels, dev_seq_lens, dev_input_mask
)
dev_dataloader = DataLoader(dev_data, batch_size=1, shuffle=False)

test_video_id = [video_struct["video_id"] for video_struct in test_SEND_features]
test_input_a_feature = torch.stack([video_struct["a_feature"] for video_struct in test_SEND_features]).float()
test_input_l_feature = torch.stack([video_struct["l_feature"] for video_struct in test_SEND_features])
test_input_l_mask = torch.stack([video_struct["l_mask"] for video_struct in test_SEND_features])
test_input_l_segment_ids = torch.stack([video_struct["l_segment_ids"] for video_struct in test_SEND_features])
test_input_v_feature = torch.stack([video_struct["v_feature"] for video_struct in test_SEND_features]).float()
test_rating_labels = torch.stack([video_struct["rating"] for video_struct in test_SEND_features]).float()
test_seq_lens = torch.tensor([[video_struct["seq_len"]] for video_struct in test_SEND_features]).float()
test_input_mask = torch.stack([video_struct["input_mask"] for video_struct in test_SEND_features])
test_data = TensorDataset(
    test_input_a_feature, 
    test_input_l_feature, test_input_l_mask, test_input_l_segment_ids,
    test_input_v_feature, test_rating_labels, test_seq_lens, test_input_mask
)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [48]:
def evaluate_ablation(
    video_id, dataloader, model, condition="A,V,L"
):
    ret = {}
    video_index = 0
    pbar = tqdm(dataloader, desc="videos")
    for step, batch in enumerate(pbar):
        vid_id = video_id[video_index]
        ret[vid_id] = {}
        # print(f"analyzing ablation studies on video_id={vid_id}")
        input_a_feature, input_l_feature, input_l_mask, input_l_segment_ids, \
            input_v_feature, rating_labels, seq_lens, input_mask = batch
        # based one condition, we need to mask out some channels!
        if "A" not in condition:
            input_a_feature = torch.zeros_like(input_a_feature)
        if "V" not in condition:
            input_v_feature = torch.zeros_like(input_v_feature)
        if "L" not in condition:
            input_l_feature = torch.zeros_like(input_l_feature)
        _, output = \
            model(input_a_feature, input_l_feature, input_l_mask, input_l_segment_ids,
                  input_v_feature, rating_labels, input_mask)
        seq_l = int(seq_lens[0].tolist()[0])
        pred = output[0][:seq_l].cpu().detach().numpy()
        true = rating_labels[0][:seq_l].cpu().detach().numpy()
        ccc = eval_ccc(pred, true)
        ret[vid_id]["pred"] = pred
        ret[vid_id]["true"] = true
        video_index += 1
    return ret

In [49]:
conditions = ["A,V,L", "A,V", "A,L", "V,L", "A", "V", "L"]
mega_results = {}
for condition in conditions:
    print("analyzing results for condition: ", condition)
    train_results = evaluate_ablation(
        train_video_id, train_dataloader, model,
        condition=condition
    )
    
    dev_results = evaluate_ablation(
        dev_video_id, dev_dataloader, model,
        condition=condition
    )

    test_results = evaluate_ablation(
        test_video_id, test_dataloader, model,
        condition=condition
    )
    mega_results[condition] = {}
    for k,v in train_results.items():
        mega_results[condition][k] = v
    for k,v in dev_results.items():
        mega_results[condition][k] = v
    for k,v in test_results.items():
        mega_results[condition][k] = v

videos:   0%|          | 0/114 [00:00<?, ?it/s]

analyzing results for condition:  A,V,L


videos:   0%|          | 0/114 [00:00<?, ?it/s]

analyzing results for condition:  A,V


videos:   0%|          | 0/114 [00:00<?, ?it/s]

...

videos:   0%|          | 0/114 [00:00<?, ?it/s]

analyzing results for condition:  V


videos:   0%|          | 0/114 [00:00<?, ?it/s]

analyzing results for condition:  L


videos: 100%|██████████| 39/39 [01:40<00:00,  2.58s/it]


In [50]:
print("output dir: ", output_dir)

output dir:  ../data-files/target/


In [51]:
# for each video, we are creating a file to save ratings for all conditions.
for video in mega_results["A,V,L"].keys():
    with open(os.path.join(output_dir, f"{video}.csv"), "w") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        headers = [c for c in conditions]
        headers += ["actual"]
        writer.writerow(headers)
        s_len = len(mega_results["A,V,L"][video]["pred"])
        for i in range(s_len): # write line by line.
            row = []
            for condition in conditions:
                norm_r = (mega_results[condition][video]["pred"][i]+1.0)/2.0
                row.append(norm_r)
            norm_r = (mega_results[condition][video]["true"][i]+1.0)/2.0
            row.append(norm_r)
            writer.writerow(row)

In [52]:
with open("../data-files/train_ids.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    headers = ["vid_id"]
    writer.writerow(headers)
    for vid_id in train_video_id:
        writer.writerow([vid_id])
with open("../data-files/dev_ids.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    headers = ["vid_id"]
    writer.writerow(headers)
    for vid_id in dev_video_id:
        writer.writerow([vid_id])
with open("../data-files/test_ids.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    headers = ["vid_id"]
    writer.writerow(headers)
    for vid_id in test_video_id:
        writer.writerow([vid_id])

#### Evaluate with Hebrew Videos

In [58]:
use_target_ratings = True

In [59]:
# loading model from saved model.
model = MultimodalEmotionPrediction()
new_state_dict = OrderedDict()
DEVICE = torch.device('cpu')   # 'cpu' in this case
if use_target_ratings:
    model_path = "../target/best_ccc_pytorch_model.bin"
else:
    model_path = "../observer/best_ccc_pytorch_model.bin"
print("loading the model from: ", model_path)
state_dict = torch.load(model_path, map_location=DEVICE)["model"]
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
_ = model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing LinguisticEncoderBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing LinguisticEncoderBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LinguisticEncoderBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


loading the model from:  ../target/best_ccc_pytorch_model.bin


In [60]:
if use_target_ratings:
    output_dir = "../data-files/target_hebrew/"
else:
    output_dir = "../data-files/observer_hebrew/"

In [61]:
def preprocess_HEBREW_files(
    data_dir, # Multitmodal X
    time_window_in_sec=4.0,
    modality_dir_map = {"acoustic": "acoustic-egemaps",  
                        "linguistic": "linguistic-word-level", # we don't load features
                        "visual": "image-raw", # image is nested,
                        "target": "observer_EWE",
                       },
    preprocess= {'acoustic': lambda df : df.loc[:,' F0semitoneFrom27.5Hz_sma3nz_amean':' equivalentSoundLevel_dBp'],
                 'acoustic_timer': lambda df : df.loc[:,' frameTime'],
                 'linguistic': lambda df : df.loc[:,'word'],
                 'linguistic_timer': lambda df : df.loc[:,'time-offset'],
                 'target': lambda df : ((df.loc[:,'evaluatorWeightedEstimate'] / 50.0) - 1.0),
                 'target_timer': lambda df : df.loc[:,'time'],
                },
    pad_symbol=0,
    max_number_of_file=-1
):
    SEND_videos = []
    
    # basically, let us gett all the video ids?
    a_ids = [f.split("_")[0]+"_"+f.split("_")[1] 
             for f in listdir(os.path.join(data_dir, modality_dir_map["acoustic"])) 
             if isfile(os.path.join(data_dir, modality_dir_map["acoustic"], f))]
    v_ids = [f.split("_")[0]+"_"+f.split("_")[1] 
             for f in listdir(os.path.join(data_dir, modality_dir_map["visual"])) 
             if f != ".DS_Store"]

    if max_number_of_file != -1:
        logger.info(f"WARNING: Only loading #{max_number_of_file} videos.")
    max_seq_len = -1
    video_count = 0
    for video_id in a_ids: # pick any one!
        if max_number_of_file != -1 and video_count >= max_number_of_file:
            break # we enforce!
        if video_count > 1 and video_count%100 == 0:
            logger.info(f"Processed #{len(SEND_videos)} videos.")
            # logger.info(SEND_videos[-1])
        
        # we need to fix this to get features aligned.
        
        # Step 1: Load rating data, and we can get window partitioned according to our interval.
        a_file = os.path.join(data_dir, modality_dir_map["acoustic"], f"{video_id}_acousticFeatures.csv")
        a_df = pd.read_csv(a_file)
        a_features = np.array(preprocess["acoustic"](a_df))
        a_timestamps = np.array(preprocess["acoustic_timer"](a_df))
        windows = []
        number_of_window = int(max(a_timestamps)//time_window_in_sec)
        for i in range(0, number_of_window):
            windows += [(i*time_window_in_sec, (i+1)*time_window_in_sec)]
        if max(a_timestamps) > (i+1)*time_window_in_sec:
            windows += [((i+1)*time_window_in_sec, max(a_timestamps))]
        # [(0, 5], (5, 10], ...]

        # acoustic features process
        a_file = os.path.join(data_dir, modality_dir_map["acoustic"], f"{video_id}_acousticFeatures.csv")
        a_df = pd.read_csv(a_file)
        a_features = np.array(preprocess["acoustic"](a_df))
        a_timestamps = np.array(preprocess["acoustic_timer"](a_df))
        a_feature_dim = a_features.shape[1]
        assert a_features.shape[0] == a_timestamps.shape[0]
        sampled_a_features_raw = [[] for i in range(len(windows))]
        for i in range(0, a_timestamps.shape[0]):
            # using mod to hash to the correct bucket.
            hash_in_window = int(a_timestamps[i]//time_window_in_sec)
            if hash_in_window >= len(windows):
                continue # we cannot predict after ratings max.
            sampled_a_features_raw[hash_in_window].append(a_features[i])
        sampled_a_features = []
        for window in sampled_a_features_raw:
            # only acoustic need to consider this I think.
            if len(window) == 0:
                collate_window = np.zeros(a_feature_dim)
            else:
                collate_window = np.mean(np.array(window), axis=0)
            sampled_a_features.append(collate_window)

        # visual features process
        # for visual, we actually need to active control what image we load, we
        # cannot just load all images, it will below memory.
        fps=30 # We may need to dynamically figure out this number?
        frame_names = []
        for f in listdir(os.path.join(data_dir, modality_dir_map["visual"], video_id)):
            if ".jpg" in f:
                frame_names += [(int(f.split("_")[0][5:])*(1.0/fps), f)]
        frame_names.sort(key=lambda x:x[0])
        sampled_v_features_raw = [[] for i in range(len(windows))]
        for f in frame_names:
            # using mod to hash to the correct bucket.
            hash_in_window = int(f[0]//time_window_in_sec)
            if hash_in_window >= len(windows):
                continue # we cannot predict after ratings max.
            sampled_v_features_raw[hash_in_window].append(f)

        sampled_v_features = []
        for window in sampled_v_features_raw:
            if len(window) == 0:
                f_data = np.zeros((224,224,3))
            else:
                # we collate by using the last frame in the time window.
                f = window[-1]
                f_path = os.path.join(data_dir, modality_dir_map["visual"], video_id, f[1])
                f_image = Image.open(f_path)
                f_data = asarray(f_image)
            sampled_v_features.append(f_data)
        
        max_window_cutoff_a = int(max(a_timestamps)//time_window_in_sec)
        max_window_cutoff_v = int(frame_names[-1][0]//time_window_in_sec)
        max_window_cutoff = min([max_window_cutoff_a, max_window_cutoff_v])
        sampled_a_features = sampled_a_features[:max_window_cutoff]
        sampled_v_features = sampled_v_features[:max_window_cutoff]
        
        video_struct = {
            "video_id": video_id,
            "a_feature": sampled_a_features,
            "v_feature": sampled_v_features,
            "seq_len": len(sampled_a_features),
            "input_mask": np.ones(len(sampled_a_features)).tolist()
        }
        video_count += 1
        SEND_videos += [video_struct]
        if len(sampled_a_features) > max_seq_len:
            max_seq_len = len(sampled_a_features)
    
    # padding based on length
    for video_struct in SEND_videos:
        for i in range(max_seq_len-video_struct["seq_len"]):
            video_struct["a_feature"].append(np.zeros(a_feature_dim))
            video_struct["v_feature"].append(np.zeros((224,224,3)))
            video_struct["input_mask"].append(0)

        video_struct["a_feature"] = torch.tensor(video_struct["a_feature"])
        video_struct["v_feature"] = torch.tensor(video_struct["v_feature"])
        video_struct["input_mask"] = torch.LongTensor(video_struct["input_mask"])
        
    return SEND_videos

In [16]:
# Loading all the data partitions.
data_dir = "../../SENDv1-data/"
test_modalities_data_dir = os.path.join(data_dir, "features/Test-Hebrew/")
test_HEBREW_features = preprocess_HEBREW_files(
    test_modalities_data_dir,
    modality_dir_map=modality_dir_map,
    preprocess=preprocess,
    max_number_of_file=-1
)

In [18]:
test_video_id = [video_struct["video_id"] for video_struct in test_HEBREW_features]
test_input_a_feature = torch.stack([video_struct["a_feature"] for video_struct in test_HEBREW_features]).float()
test_input_v_feature = torch.stack([video_struct["v_feature"] for video_struct in test_HEBREW_features]).float()
test_seq_lens = torch.tensor([[video_struct["seq_len"]] for video_struct in test_HEBREW_features]).float()
test_input_mask = torch.stack([video_struct["input_mask"] for video_struct in test_HEBREW_features])
test_data = TensorDataset(
    test_input_a_feature, 
    test_input_v_feature, 
    test_seq_lens, test_input_mask
)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [62]:
def evaluate_ablation(
    video_id, dataloader, model, condition="A,V"
):
    ret = {}
    video_index = 0
    pbar = tqdm(dataloader, desc="videos")
    for step, batch in enumerate(pbar):
        vid_id = video_id[video_index]
        ret[vid_id] = {}
        # print(f"analyzing ablation studies on video_id={vid_id}")
        input_a_feature, input_v_feature, seq_lens, input_mask = batch        
        
        # based one condition, we need to mask out some channels!
        if "A" not in condition:
            input_a_feature = torch.zeros_like(input_a_feature)
        if "V" not in condition:
            input_v_feature = torch.zeros_like(input_v_feature)

        # mock linguistic and rating data.
        batch = input_a_feature.shape[0]
        seq_l = input_a_feature.shape[1]
        input_l_feature = torch.zeros((batch, seq_l, 3)).long()
        input_l_mask = torch.ones((batch, seq_l, 3)).long()
        input_l_segment_ids = torch.zeros((batch, seq_l, 3)).long()
        rating_labels = torch.zeros((batch, seq_l))
            
        _, output = \
            model(input_a_feature, input_l_feature, input_l_mask, input_l_segment_ids,
                  input_v_feature, rating_labels, input_mask)
        seq_l = int(seq_lens[0].tolist()[0])
        pred = output[0][:seq_l].cpu().detach().numpy()
        true = rating_labels[0][:seq_l].cpu().detach().numpy()
        ccc = eval_ccc(pred, true)
        ret[vid_id]["pred"] = pred
        ret[vid_id]["true"] = true
        video_index += 1
    return ret

In [63]:
mega_results = {}

In [64]:
conditions = ["A,V", "A", "V",]
for condition in conditions:
    print("analyzing results for condition: ", condition)

    test_results = evaluate_ablation(
        test_video_id, test_dataloader, model,
        condition=condition
    )
    mega_results[condition] = {}
    for k,v in test_results.items():
        mega_results[condition][k] = v

videos:   0%|          | 0/9 [00:00<?, ?it/s]

analyzing results for condition:  A,V


videos:   0%|          | 0/9 [00:00<?, ?it/s]

analyzing results for condition:  A


videos:   0%|          | 0/9 [00:00<?, ?it/s]

analyzing results for condition:  V


videos: 100%|██████████| 9/9 [03:04<00:00, 20.47s/it]


In [65]:
print("output dir: ", output_dir)

output dir:  ../data-files/target_hebrew/


In [57]:
# for each video, we are creating a file to save ratings for all conditions.
conditions = ["A,V", "A", "V",]
for video in mega_results["A,V"].keys():
    with open(os.path.join(output_dir, f"{video}.csv"), "w") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        headers = [c for c in conditions]
        writer.writerow(headers)
        s_len = len(mega_results["A,V"][video]["pred"])
        for i in range(s_len): # write line by line.
            row = []
            for condition in conditions:
                norm_r = (mega_results[condition][video]["pred"][i]+1.0)/2.0
                row.append(norm_r)
            writer.writerow(row)